# Data Preprocessing Tiempos

## Libraries 

In [1]:
import pandas as pd 
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import json


## Modalidades

### Loading Data

In [2]:
df_tiempos_pacientes = pd.read_excel("Tiempos Pacientes.xlsx")

### First Elements in DF

In [3]:
df_tiempos_pacientes.head()

,Sucursal,EstudioModalidad,ReservacionSP,TEPFechaEspera,TEPHoraInicio,TEPHoraFin,TEPMinutos,TAPFechaAtencion,TAPHoraInicio,TAPHoraFin,TAPMinutos,PacienteID,PacienteSP,PacienteGenero,PacienteFechaNacimiento,PacienteCodigoPostal
0,COYOACAN,DENSITOMETRIA,806115606,20250302,2025-03-02 13:20:00.000,2025-03-02 11:40:15.113,0.00,20250302,2025-03-02 11:40:15.113,2025-03-02 11:47:18.170,7.05,17256328.0,17520850.0,FEMENINO,1971-07-24,4310.0
1,COYOACAN,DENSITOMETRIA,918870299,20250329,2025-03-29 07:11:15.770,2025-03-29 07:39:49.400,28.57,20250329,2025-03-29 07:39:49.400,2025-03-29 07:45:18.230,5.48,46138407.0,9225063.0,FEMENINO,1967-02-08,3570.0
2,COYOACAN,DENSITOMETRIA,921436045,20250321,2025-03-21 15:55:53.723,2025-03-21 16:15:07.230,19.23,20250321,2025-03-21 16:15:07.230,2025-03-21 16:21:50.127,6.72,41262646.0,4138508.0,FEMENINO,1948-09-23,3240.0
3,COYOACAN,DENSITOMETRIA,923595474,20250301,2025-03-01 08:03:01.360,2025-03-01 09:42:16.990,99.25,20250301,2025-03-01 09:42:16.990,2025-03-01 09:48:03.577,5.78,48597444.0,11686051.0,FEMENINO,1976-04-07,1410.0
4,COYOACAN,DENSITOMETRIA,926170090,20250302,2025-03-02 09:40:00.000,2025-03-02 09:20:46.240,0.00,20250302,2025-03-02 09:20:46.240,2025-03-02 09:27:47.617,7.02,63749589.0,47710075.0,FEMENINO,1978-12-28,4380.0


### Creating a copy to work on

In [4]:
df_copy = df_tiempos_pacientes.copy()

### Checking Columns

In [6]:
df_copy.columns

Index(['Sucursal', 'EstudioModalidad', 'ReservacionSP', 'TEPFechaEspera',
       'TEPHoraInicio', 'TEPHoraFin', 'TEPMinutos', 'TAPFechaAtencion',
       'TAPHoraInicio', 'TAPHoraFin', 'TAPMinutos', 'PacienteID', 'PacienteSP',
       'PacienteGenero', 'PacienteFechaNacimiento', 'PacienteCodigoPostal'],
      dtype='object')

### Droping Unecessary Columns

In [7]:
df_copy.drop(columns = ["ReservacionSP", "PacienteCodigoPostal"], inplace=True)

### Droping Duplicates

In [8]:
df_copy.drop_duplicates(inplace=True)

### Fixing Bad calculations

In [9]:
### Arreglando datos donde TEPHoraInicio tiene una hora posterior a TEPHoraFin

# Identificar los casos donde la hora de inicio es posterior a la hora de fin
# Esto crea una máscara booleana que será True para los registros con tiempos invertidos
mask_tiempos_invertidos = df_copy['TEPHoraInicio'] > df_copy['TEPHoraFin']

# Guardar temporalmente los valores de hora de inicio para los casos con tiempos invertidos
tmp = df_copy.loc[mask_tiempos_invertidos, 'TEPHoraInicio'].copy()

# Intercambiar los valores: la hora de inicio ahora será la hora de fin
df_copy.loc[mask_tiempos_invertidos, 'TEPHoraInicio'] = df_copy.loc[mask_tiempos_invertidos, 'TEPHoraFin']

# La hora de fin ahora será el valor original de la hora de inicio (guardado en tmp)
df_copy.loc[mask_tiempos_invertidos, 'TEPHoraFin'] = tmp

# Asignar TAPHoraInicio igual a TEPHoraFin en los registros afectados
df_copy.loc[mask_tiempos_invertidos, 'TAPHoraInicio'] = df_copy.loc[mask_tiempos_invertidos, 'TEPHoraFin']



In [10]:
# Cálculo de los minutos
df_copy.loc[mask_tiempos_invertidos, 'TEPMinutos'] = (df_copy.loc[mask_tiempos_invertidos, 'TEPHoraFin'] - df_copy.loc[mask_tiempos_invertidos, 'TEPHoraInicio']).dt.total_seconds() / 60

In [11]:
df_copy.loc[mask_tiempos_invertidos, 'TAPHoraFin'] = df_copy.loc[mask_tiempos_invertidos, 'TAPHoraInicio'] + pd.to_timedelta(df_copy.loc[mask_tiempos_invertidos, 'TAPMinutos'], unit='m')

In [12]:
### Arreglando datos donde TAPHoraInicio tiene una hora posterior a TAPHoraFin

# Identificar los casos donde la hora de inicio es posterior a la hora de fin
# Esto crea una máscara booleana que será True para los registros con tiempos invertidos
mask_tiempos_invertidos = df_copy['TAPHoraInicio'] > df_copy['TAPHoraFin']

# Guardar temporalmente los valores de hora de inicio para los casos con tiempos invertidos
tmp = df_copy.loc[mask_tiempos_invertidos, 'TAPHoraInicio'].copy()

# Intercambiar los valores: la hora de inicio ahora será la hora de fin
df_copy.loc[mask_tiempos_invertidos, 'TAPHoraInicio'] = df_copy.loc[mask_tiempos_invertidos, 'TAPHoraFin']

# La hora de fin ahora será el valor original de la hora de inicio (guardado en tmp)
df_copy.loc[mask_tiempos_invertidos, 'TAPHoraFin'] = tmp

In [13]:
df_copy.loc[mask_tiempos_invertidos, 'TAPMinutos'] = (df_copy.loc[mask_tiempos_invertidos, 'TAPHoraFin'] - df_copy.loc[mask_tiempos_invertidos, 'TAPHoraInicio']).dt.total_seconds() / 60

In [14]:
### Arreglando datos con TEPHoraInicio con año 1900

# Identificar registros donde TEPHoraInicio tiene el año 1900
mask_1900 = df_copy['TEPHoraInicio'].dt.year == 1900

# Comparar solo la hora de TEPHoraInicio con la hora de TEPHoraFin
mask_hora_invertida = df_copy.loc[mask_1900, 'TEPHoraInicio'].dt.time > df_copy.loc[mask_1900, 'TEPHoraFin'].dt.time

# Intercambiar los valores de TEPHoraInicio y TEPHoraFin en los casos donde la hora de inicio es mayor
tmp = df_copy.loc[mask_1900 & mask_hora_invertida, 'TEPHoraInicio'].copy()
df_copy.loc[mask_1900 & mask_hora_invertida, 'TEPHoraInicio'] = df_copy.loc[mask_1900 & mask_hora_invertida, 'TEPHoraFin']
df_copy.loc[mask_1900 & mask_hora_invertida, 'TEPHoraFin'] = tmp

# Recalcular TAPHoraInicio para los registros afectados
df_copy.loc[mask_1900 & mask_hora_invertida, 'TAPHoraInicio'] = df_copy.loc[mask_1900 & mask_hora_invertida, 'TEPHoraFin']

In [15]:
# Encuentra las filas donde el año de TEPHoraFin es 1900
mask = df_copy['TEPHoraFin'].dt.year == 1900

df_copy.loc[mask, 'TEPHoraFin'] = df_copy.loc[mask].apply(
    lambda row: row['TEPHoraInicio'].replace(
        hour=row['TEPHoraFin'].hour,
        minute=row['TEPHoraFin'].minute,
        second=row['TEPHoraFin'].second,
        microsecond=row['TEPHoraFin'].microsecond
    ),
    axis=1
)


# Reemplaza la fecha completa por la de TEPHoraFin, conservando la hora y minutos de TEPHoraInicio

mask = df_copy['TEPHoraInicio'].dt.year == 1900

df_copy.loc[mask, 'TEPHoraInicio'] = df_copy.loc[mask].apply(
    lambda row: row['TEPHoraFin'].replace(
        hour=row['TEPHoraInicio'].hour,
        minute=row['TEPHoraInicio'].minute,
        second=row['TEPHoraInicio'].second,
        microsecond=row['TEPHoraInicio'].microsecond
    ),
    axis=1
)


In [16]:
# Encuentra las filas donde el año de TAPHoraFin es 1900
mask = df_copy['TAPHoraFin'].dt.year == 1900

df_copy.loc[mask, 'TAPHoraFin'] = df_copy.loc[mask].apply(
    lambda row: row['TAPHoraInicio'].replace(
        hour=row['TAPHoraFin'].hour,
        minute=row['TAPHoraFin'].minute,
        second=row['TAPHoraFin'].second,
        microsecond=row['TAPHoraFin'].microsecond
    ),
    axis=1
)

# Reemplaza la fecha completa por la de TAPHoraFin, conservando la hora y minutos de TAPHoraInicio
mask = df_copy['TAPHoraInicio'].dt.year == 1900

df_copy.loc[mask, 'TAPHoraInicio'] = df_copy.loc[mask].apply(
    lambda row: row['TAPHoraFin'].replace(
        hour=row['TAPHoraInicio'].hour,
        minute=row['TAPHoraInicio'].minute,
        second=row['TAPHoraInicio'].second,
        microsecond=row['TAPHoraInicio'].microsecond
    ),
    axis=1
)

In [17]:
# Encuentra los casos donde TEPHoraFin > TAPHoraInicio (o el criterio que determines)
mask = df_copy["TEPHoraFin"] > df_copy["TAPHoraInicio"]

def ordenar_y_recalcular(row):
    # Toma los cuatro timestamps
    tiempos = [
        row["TEPHoraInicio"],
        row["TEPHoraFin"],
        row["TAPHoraInicio"],
        row["TAPHoraFin"]
    ]
    # Ordena de menor a mayor
    tiempos_ordenados = sorted(tiempos)
    # Los dos del medio deben ser iguales
    tiempos_ordenados[1] = tiempos_ordenados[2] = tiempos_ordenados[1]
    # Asigna de vuelta
    row["TEPHoraInicio"] = tiempos_ordenados[0]
    row["TEPHoraFin"] = tiempos_ordenados[1]
    row["TAPHoraInicio"] = tiempos_ordenados[2]
    row["TAPHoraFin"] = tiempos_ordenados[3]
    # Recalcula los minutos
    row["TEPMinutos"] = (row["TEPHoraFin"] - row["TEPHoraInicio"]).total_seconds() / 60
    row["TAPMinutos"] = (row["TAPHoraFin"] - row["TAPHoraInicio"]).total_seconds() / 60
    return row

# Aplica la función solo a las filas que cumplen el criterio
df_copy.loc[mask] = df_copy.loc[mask].apply(ordenar_y_recalcular, axis=1)

In [18]:
df_copy[df_copy["TEPHoraFin"] != df_copy["TAPHoraInicio"]].isnull().sum()

Sucursal                      0
EstudioModalidad              0
TEPFechaEspera                0
TEPHoraInicio                32
TEPHoraFin                 1611
TEPMinutos                    0
TAPFechaAtencion              0
TAPHoraInicio              1612
TAPHoraFin                    0
TAPMinutos                 1612
PacienteID                    0
PacienteSP                    0
PacienteGenero                0
PacienteFechaNacimiento       0
dtype: int64

In [19]:
### Testear si los datos se han arreglado

print(df_copy[df_copy["TEPHoraInicio"] > df_copy["TEPHoraFin"]].shape[0] == 0)
print(df_copy[df_copy["TAPHoraInicio"] > df_copy["TAPHoraFin"]].shape[0] == 0)
print(df_copy[df_copy["TEPHoraFin"] == df_copy["TAPHoraInicio"]].shape[0])
print(df_copy[df_copy["TEPMinutos"] < 0].shape[0] == 0)
print(df_copy[df_copy["TAPMinutos"] < 0].shape[0] == 0)
print(df_copy[df_copy["TAPHoraInicio"] == df_copy["TEPHoraInicio"]].shape[0] == 1)

print((df_copy['TEPHoraInicio'].dt.year == 1900).sum())


True
True
105631
True
True
True
0


### Max number of studies per "sucursal"

In [20]:
df_analisis = df_copy.copy()

# Si la columna es tipo int o string, conviértela a datetime
df_analisis['TEPFechaEspera'] = pd.to_datetime(df_analisis['TEPFechaEspera'], format='%Y%m%d')

In [21]:
# Agrupa por paciente, sucursal y fecha, y cuenta los estudios
conteo = df_analisis.groupby(['PacienteID', 'Sucursal', 'TEPFechaEspera']).size().reset_index(name='num_estudios')

# Ahora, para cada sucursal, encuentra el máximo número de estudios hechos por un paciente en un día
maximos_por_sucursal = conteo.groupby('Sucursal')['num_estudios'].max().reset_index()

print(maximos_por_sucursal)

                   Sucursal  num_estudios
0                  COYOACAN            10
1                  CULIACAN             7
2          CULIACAN CAÑADAS             8
3  CULIACAN COLEGIO MILITAR             8
4     CULIACAN LA CONQUISTA             4
5       CULIACAN LAS TORRES             7
6         CULIACAN NAKAYAMA             5
7   CULIACAN UNIVERSITARIOS             8


In [22]:
conteo.sort_values(by = "num_estudios", ascending = False)

,PacienteID,Sucursal,TEPFechaEspera,num_estudios
60437,65424279.0,COYOACAN,2025-03-14,10
61672,65490573.0,COYOACAN,2025-03-19,9
41933,47525412.0,COYOACAN,2025-03-01,9
36586,42625626.0,COYOACAN,2025-03-02,9
45071,50544398.0,COYOACAN,2025-03-03,8
...,...,...,...,...
25191,38219630.0,CULIACAN,2025-03-13,1
25192,38220190.0,CULIACAN,2025-03-15,1
25193,38220190.0,CULIACAN LA CONQUISTA,2025-03-04,1
25195,38220781.0,CULIACAN NAKAYAMA,2025-03-01,1


### Checking and treating Null Values

In [23]:
df_copy_null = df_copy.copy()

In [24]:
df_copy_null.isnull().sum()

Sucursal                       0
EstudioModalidad               0
TEPFechaEspera                 0
TEPHoraInicio                233
TEPHoraFin                  1611
TEPMinutos                     0
TAPFechaAtencion               0
TAPHoraInicio               1612
TAPHoraFin                  1452
TAPMinutos                  3064
PacienteID                 14508
PacienteSP                 14508
PacienteGenero             14508
PacienteFechaNacimiento    14508
dtype: int64

In [25]:
# Eliminar registros donde TEPHoraFin y TAPHoraInicio son nulos simultáneamente
df_copy_null = df_copy_null[
    ~(df_copy_null['TEPHoraFin'].isnull() & 
      df_copy_null['TAPHoraInicio'].isnull())
]

# Verificar cuántos registros quedaron
print(f"Número de registros después de eliminar los nulos simultáneos: {len(df_copy_null)}")

Número de registros después de eliminar los nulos simultáneos: 105665


In [26]:
df_copy_null.isnull().sum()

Sucursal                       0
EstudioModalidad               0
TEPFechaEspera                 0
TEPHoraInicio                233
TEPHoraFin                     0
TEPMinutos                     0
TAPFechaAtencion               0
TAPHoraInicio                  1
TAPHoraFin                  1452
TAPMinutos                  1453
PacienteID                 14508
PacienteSP                 14508
PacienteGenero             14508
PacienteFechaNacimiento    14508
dtype: int64

In [27]:
def remove_outliers(df, column):
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    return df[(df[column] >= lower_bound) & (df[column] <= upper_bound)]

# Crear una copia del DataFrame para no modificar el original
df_sin_outliers = df_copy.copy()

# Remover outliers de TEPMinutos y TAPMinutos
df_sin_outliers = remove_outliers(df_sin_outliers, 'TEPMinutos')
df_sin_outliers = remove_outliers(df_sin_outliers, 'TAPMinutos')

# Calcular las medias sin outliers
medias_sin_outliers = df_sin_outliers.groupby(['Sucursal', 'EstudioModalidad']).agg({
    'TEPMinutos': 'mean',
    'TAPMinutos': 'mean'
}).reset_index()



In [28]:
medias_sin_outliers

,Sucursal,EstudioModalidad,TEPMinutos,TAPMinutos
0,COYOACAN,DENSITOMETRIA,15.400760,6.610322
1,COYOACAN,ELECTROCARDIOGRAMA,11.174525,6.690047
2,COYOACAN,LABORATORIO,3.600297,3.164254
3,COYOACAN,MASTOGRAFIA,13.893288,8.694221
4,COYOACAN,NUTRICION,9.008350,10.757314
...,...,...,...,...
62,CULIACAN UNIVERSITARIOS,NUTRICION,6.183625,14.408788
63,CULIACAN UNIVERSITARIOS,OPTOMETRIA,5.665942,8.877630
64,CULIACAN UNIVERSITARIOS,PAPANICOLAOU,8.380141,6.809195
65,CULIACAN UNIVERSITARIOS,RAYOS X,8.347880,4.602209


In [29]:
### Imputacion en valores nulos en TEPHoraFin
# Primero, creamos un diccionario con las medias para facilitar el acceso
medias_dict = medias_sin_outliers.set_index(['Sucursal', 'EstudioModalidad']).to_dict('index')

In [30]:
def imputar_tapminutos(row):
    if pd.isnull(row["TAPMinutos"]):
        media = medias_dict.get((row["Sucursal"], row["EstudioModalidad"]), {}).get("TAPMinutos", 0)
        return media
    return row["TAPMinutos"]

df_copy_null["TAPMinutos"] = df_copy_null.apply(imputar_tapminutos, axis=1)

In [31]:
# Recalcular TAPHoraFin solo cuando sea nulo
mask_tephorafin_nulo = df_copy_null["TAPHoraFin"].isnull()
df_copy_null.loc[mask_tephorafin_nulo, "TAPHoraFin"] = df_copy_null.loc[mask_tephorafin_nulo, "TAPHoraInicio"] + pd.to_timedelta(df_copy_null.loc[mask_tephorafin_nulo, "TAPMinutos"], unit="minutes")

# Recalcular TAPHoraInicio solo cuando sea nulo
mask_tephorainicio_nulo = df_copy_null["TAPHoraInicio"].isnull()
df_copy_null.loc[mask_tephorainicio_nulo, "TAPHoraInicio"] = df_copy_null.loc[mask_tephorainicio_nulo, "TAPHoraFin"] - pd.to_timedelta(df_copy_null.loc[mask_tephorainicio_nulo, "TAPMinutos"], unit="minutes")

In [32]:
df_copy_null.isnull().sum()

Sucursal                       0
EstudioModalidad               0
TEPFechaEspera                 0
TEPHoraInicio                233
TEPHoraFin                     0
TEPMinutos                     0
TAPFechaAtencion               0
TAPHoraInicio                  0
TAPHoraFin                     0
TAPMinutos                     0
PacienteID                 14508
PacienteSP                 14508
PacienteGenero             14508
PacienteFechaNacimiento    14508
dtype: int64

In [33]:
def imputar_tepminutos(row):
    if pd.isnull(row["TEPHoraInicio"]) or pd.isnull(row["TEPHoraFin"]):

        media = medias_dict.get((row["Sucursal"], row["EstudioModalidad"]), {}).get("TEPMinutos", 0)
        return media
    return row["TEPMinutos"]

df_copy_null["TEPMinutos"] = df_copy_null.apply(imputar_tepminutos, axis=1)

In [34]:
def imputar_tephorainicio(row):
    if pd.isnull(row["TEPHoraInicio"]):
        return row["TEPHoraFin"] - pd.to_timedelta(row["TEPMinutos"], unit="minutes")
    return row["TEPHoraInicio"]

df_copy_null["TEPHoraInicio"] = df_copy_null.apply(imputar_tephorainicio, axis=1)


In [35]:
df_copy_null.isnull().sum()


Sucursal                       0
EstudioModalidad               0
TEPFechaEspera                 0
TEPHoraInicio                  0
TEPHoraFin                     0
TEPMinutos                     0
TAPFechaAtencion               0
TAPHoraInicio                  0
TAPHoraFin                     0
TAPMinutos                     0
PacienteID                 14508
PacienteSP                 14508
PacienteGenero             14508
PacienteFechaNacimiento    14508
dtype: int64

In [36]:
df_copy_null

,Sucursal,EstudioModalidad,TEPFechaEspera,TEPHoraInicio,TEPHoraFin,TEPMinutos,TAPFechaAtencion,TAPHoraInicio,TAPHoraFin,TAPMinutos,PacienteID,PacienteSP,PacienteGenero,PacienteFechaNacimiento
0,COYOACAN,DENSITOMETRIA,20250302,2025-03-02 11:40:15.113,2025-03-02 13:20:00.000,99.748117,20250302,2025-03-02 13:20:00.000,2025-03-02 13:27:03.000,7.05,17256328.0,17520850.0,FEMENINO,1971-07-24
1,COYOACAN,DENSITOMETRIA,20250329,2025-03-29 07:11:15.770,2025-03-29 07:39:49.400,28.570000,20250329,2025-03-29 07:39:49.400,2025-03-29 07:45:18.230,5.48,46138407.0,9225063.0,FEMENINO,1967-02-08
2,COYOACAN,DENSITOMETRIA,20250321,2025-03-21 15:55:53.723,2025-03-21 16:15:07.230,19.230000,20250321,2025-03-21 16:15:07.230,2025-03-21 16:21:50.127,6.72,41262646.0,4138508.0,FEMENINO,1948-09-23
3,COYOACAN,DENSITOMETRIA,20250301,2025-03-01 08:03:01.360,2025-03-01 09:42:16.990,99.250000,20250301,2025-03-01 09:42:16.990,2025-03-01 09:48:03.577,5.78,48597444.0,11686051.0,FEMENINO,1976-04-07
4,COYOACAN,DENSITOMETRIA,20250302,2025-03-02 09:20:46.240,2025-03-02 09:40:00.000,19.229333,20250302,2025-03-02 09:40:00.000,2025-03-02 09:47:01.200,7.02,63749589.0,47710075.0,FEMENINO,1978-12-28
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
109159,CULIACAN UNIVERSITARIOS,ULTRASONIDO,20250331,2025-03-31 14:15:13.617,2025-03-31 15:30:00.000,74.773050,20250331,2025-03-31 15:30:00.000,2025-03-31 15:36:34.800,6.58,65655850.0,49673304.0,FEMENINO,1945-10-05
109160,CULIACAN UNIVERSITARIOS,ULTRASONIDO,20250331,2025-03-31 14:16:51.950,2025-03-31 14:21:41.720,4.830000,20250331,2025-03-31 14:21:41.720,2025-03-31 14:39:08.040,17.45,43677862.0,6683966.0,FEMENINO,1974-10-25
109161,CULIACAN UNIVERSITARIOS,ULTRASONIDO,20250331,2025-03-31 14:42:47.717,2025-03-31 14:45:00.000,2.204717,20250331,2025-03-31 14:45:00.000,2025-03-31 14:58:09.000,13.15,39631600.0,2500982.0,FEMENINO,1994-10-11
109162,CULIACAN UNIVERSITARIOS,ULTRASONIDO,20250331,2025-03-31 14:40:03.463,2025-03-31 14:45:00.000,4.942283,20250331,2025-03-31 14:45:00.000,2025-03-31 14:59:00.000,14.00,24634844.0,24587950.0,FEMENINO,1997-03-11


In [37]:
### Testear que la integridad de los datos se ha mantenido

print(df_copy_null[df_copy_null["TEPHoraInicio"] > df_copy_null["TEPHoraFin"]].shape[0] == 0)
print(df_copy_null[df_copy_null["TAPHoraInicio"] > df_copy_null["TAPHoraFin"]].shape[0] == 0)
print(df_copy_null[df_copy_null["TEPHoraFin"] == df_copy_null["TAPHoraInicio"]].shape[0])
print(df_copy_null[df_copy_null["TEPMinutos"] < 0].shape[0] == 0)
print(df_copy_null[df_copy_null["TAPMinutos"] < 0].shape[0] == 0)
print(df_copy_null[df_copy_null["TAPHoraInicio"] == df_copy_null["TEPHoraInicio"]].shape[0] == 1)
print((df_copy_null['TEPHoraInicio'].dt.year == 1900).sum())

True
True
105631
True
True
True
0


## Saving Cleaned Data

In [38]:
### Guardando datos limpios
# Guardar el DataFrame en un archivo Excel

df_copy_null.to_excel('Tiempos Pacientes Limpios.xlsx', index=False)
medias_sin_outliers.to_excel("medias_por_estudio_sucursal.xlsx")